**Extracting Summary and Dynamic Features from Last.fm User History**

### Import Libraries

In [ ]:
#Importing neccessary libraries
import pickle
import calendar
from datetime import datetime
import csv
import os 
import pandas as pd
import numpy as np
from scipy import stats

### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Changes needed for Execution for Pre Covid and During Covid

* Modify acoustic_dbfile 
* Modify folder paths user_history_folder, session_folder and track_folder
* Update  curr_dt = datetime.strptime(row['datetime'],'%d %b %Y, %H:%M') --
for Durcovid '%d %b %Y, %H:%M' or for Precovid '%Y-%m-%d %H:%M:%S'.
* Update folder name while saving all_user_summary, all_user_autocorr and all_user_variability.






            


### Read pickle file containing acoustic features for all tracks.

In [ ]:
acoustic_dbfile = open('/content/drive/MyDrive/MMT_Project_Data/DurCovidData/durcovid_acoustic_file.pkl', 'rb') 
#acoustic_dbfile = open('/content/drive/MyDrive/MMT_Project_Data/PreCovidData/acoustic_11d_final.pkl', 'rb')    
acoustic_db_dict = pickle.load(acoustic_dbfile)

In [ ]:
#count = 572570
for keys in acoustic_db_dict:
    print(keys,acoustic_db_dict[keys])
    break

('Oscillate Wildly - 2011 Remaster', 'The Smiths') [0.657, 0.791, 9, -5.996, 0, 0.0278, 0.0205, 0.298, 0.101, 0.783, 116.329]


### Function to get quadrant for QPS.

In [ ]:
def get_quadrant(arousal,valence):
    #arousal = float(arousal)
    #valence = float(valence)
    if(arousal>0.5):
        if(valence>0.5):
            return 1
        else:
            return 2
    else:
        if(valence>0.5):
            return 4
        else:
            return 3
        

### Declaring folders and generating session and track files for each user. Also generate QPS for each user session.  

In [ ]:
user_history_folder = '/content/drive/MyDrive/MMT_Project_Data/DurCovidData/hashed_listening_history'
session_folder = '/content/drive/MyDrive/MMT_Project_Data/DurCovidData/sessions/'
track_folder = '/content/drive/MyDrive/MMT_Project_Data/DurCovidData/tracks/'

In [ ]:
count=0
for file in os.listdir(user_history_folder):
    count+=1
    print(f'Processing file {count}: {file}...')
    userHistory = pd.read_csv(f'{user_history_folder}/{file}')
    #reverse the df to generate sessions on time
    userHistory = userHistory[::-1]
    session_id = 1
    last_dt = None
    start= None
    user_summary_features = pd.DataFrame(columns=['sessionID','start','end','danceability','loudness','mode',
                                                  'speechiness','acousticness','instrumentalness','liveness','tempo',
                                                  'QPS_Q1','QPS_Q2','QPS_Q3','QPS_Q4','total_tracks'])
    
    user_tracks = pd.DataFrame(columns=['track','artist','frequency','sessionID'])
    #DEKLoMSAILiVT
    indices = [0,3,4,5,6,7,8,10]
    for index,row in userHistory.iterrows():  
        artist = row['artist']
        track  = row['track']
        #print(artist,track)
        #Track accoustic features present
        if (track,artist) in acoustic_db_dict:
          # Precovid '%Y-%m-%d %H:%M:%S'
          # Durcovid '%d %b %Y, %H:%M'
            curr_dt = datetime.strptime(row['datetime'],'%d %b %Y, %H:%M')
            if(start is not None):              
                time_diff = curr_dt-last_dt          
                #check for session
                if(time_diff.total_seconds()> 120*60):
                    #end=last_dt
                    session_feature_vector = session_feature_vector/session_total_tracks
                    QPS = QPS/session_total_tracks
                    entry = np.concatenate((np.array([session_id,start,last_dt]),session_feature_vector,QPS,np.array([session_total_tracks])), axis=0)
                    #user_summary_features.append(pd.DataFrame(entry).T)
                    user_summary_features.loc[len(user_summary_features)] = entry
                    #convert dictionary and store data to user tracks
                    track_data = [[i,j]for i,j in session_dict.keys()]
                    track_freq = [[i] for i in session_dict.values()] 
                    temp = np.full((len(track_data),1), session_id)
                    track_data=np.concatenate((track_data, track_freq), axis=1)
                    track_data=np.concatenate((track_data, temp), axis=1)
                    user_tracks=user_tracks.append(pd.DataFrame(track_data.tolist(),columns=user_tracks.columns.values),ignore_index=True)

                    #starting a new session
                    start = curr_dt
                    QPS = np.zeros(4)
                    session_total_tracks = 0
                    session_feature_vector = np.zeros(8)
                    session_id+=1
                    session_dict = {}
                    #DEKLoMSAILiVT
                    #arousal = features[1] #energy
                    #valence = features[9]    
            else:
                #starting a new session
                start = curr_dt
                QPS = np.zeros(4)
                session_total_tracks = 0
                session_feature_vector = np.zeros(8)
                session_dict = {}
            if ((track,artist) in session_dict):
                session_dict[(track,artist)]+=1
            else:
                session_dict[(track,artist)]=1
            last_dt = curr_dt    
            features = acoustic_db_dict[(track,artist)]
            session_feature_vector += np.array([features[i] for i in indices]) 
            session_total_tracks+= 1 
            q = get_quadrant(features[1],features[9])
            QPS[q-1] += 1
    
    #save files 
    user_name = file.split('.')[0]
    user_summary_features.to_csv(f'{session_folder}/{user_name}_session.csv',index=False,encoding='utf-8')
    user_tracks.to_csv(f'{track_folder}/{user_name}_tracks.csv',index=False,encoding='utf-8')

Processing file 1: 05157feb07f9e780637ffb96ae549365.csv...
Processing file 2: 06bd850533a713bb7800766860dc6e6e.csv...
Processing file 3: 03e7c2869742ac3b8b04a45675ab5b24.csv...
Processing file 4: 0538da9949fdaa2da6e5f18c0a27e146.csv...
Processing file 5: 03c38064fb423b47d5292bffc09785cc.csv...
Processing file 6: 080718a2684221573b174563e37c8f42.csv...
Processing file 7: 05110aecc712b5c4d62efc592507dcce.csv...
Processing file 8: 082ae734bd0f8399771b841e3f19f8a9.csv...
Processing file 9: 056d099110249e4dfc60cfc330f59f76.csv...
Processing file 10: 03a050d649de4bf6e489a064515e2f43.csv...
Processing file 11: 0274a823eec1e677645b20883a5e7897.csv...
Processing file 12: 04e3becfa6b8843aabcfed4e128ffff5.csv...
Processing file 13: 009327643adf158a3693b76e2edde04e.csv...
Processing file 14: 0820db544692f5180aeea5b7233c0f43.csv...
Processing file 15: 026c353028eefcd77442b35ce99832e0.csv...
Processing file 16: 0411187ec1298cca816ce3d255382242.csv...
Processing file 17: 05ed17d0ecc78208ba5d6a3661a9f

In [ ]:
user_summary_features.head()

,sessionID,start,end,danceability,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,QPS_Q1,QPS_Q2,QPS_Q3,QPS_Q4,total_tracks
0,1,2020-04-01 06:32:00,2020-04-01 15:03:00,0.343628,-5.644534,0.431034,0.151176,0.077087,0.435006,0.250300,129.820241,0.017241,0.913793,0.068966,0.0,58
1,2,2020-04-02 06:40:00,2020-04-02 19:27:00,0.335375,-5.437972,0.722222,0.130881,0.051334,0.428616,0.253817,129.809444,0.069444,0.875000,0.055556,0.0,72
2,3,2020-04-02 22:52:00,2020-04-02 22:59:00,0.336000,-5.552333,0.666667,0.080100,0.000007,0.644667,0.228000,119.932333,0.000000,1.000000,0.000000,0.0,3
3,4,2020-04-03 06:39:00,2020-04-03 17:24:00,0.379000,-6.265706,0.676471,0.104015,0.040111,0.672054,0.221585,123.802941,0.058824,0.882353,0.058824,0.0,34
4,5,2020-04-03 20:15:00,2020-04-03 21:02:00,0.329111,-4.014111,0.444444,0.146078,0.001747,0.489056,0.204411,136.917556,0.000000,1.000000,0.000000,0.0,9


### Calculate Repetitive Index and average of each features per user in a single file for all users and save it in csv.

In [ ]:
user_info = pd.read_csv('/content/drive/MyDrive/MMT_Project_Data/user_info.csv')

In [ ]:
count=0
all_user_summary = pd.DataFrame(columns=['user_id','K10','total_session','total_tracks','repetitiveness_index','danceability',
                                         'loudness','mode','speechiness','acousticness','instrumentalness','liveness',
                                         'tempo','QPS_Q1','QPS_Q2','QPS_Q3','QPS_Q4'])
for file in os.listdir(session_folder):
    count+=1
    print(f'Processing file {count}: {file}...')
    user_name = file.split('_')[0]
    session_df = pd.read_csv(f'{session_folder}/{user_name}_session.csv',encoding='utf-8')
    track_df = pd.read_csv(f'{track_folder}/{user_name}_tracks.csv',encoding='utf-8')
    
    total_session = session_df.shape[0]
    total_tracks = 0
    avg_features = np.zeros(12)
    for index,row in session_df.iterrows():
        session_tracks = row['total_tracks']
        avg_features= avg_features+(row.iloc[3:15]*session_tracks)
        total_tracks +=session_tracks
        #print(session_tracks)
    avg_features= avg_features/total_tracks
    #print(avg_features)
    #Code For RI
    temp=0
    session_id=1
    session_unique_songs=0
    session_result=0
    for index,row in track_df.iterrows():
        if (session_id!=row['sessionID']):
            session_id+=1
            session_result += temp/session_unique_songs
            temp=0
            session_unique_songs=0

        freq = row['frequency']
        temp +=freq * (1 if freq >= 2 else 0)
        session_unique_songs +=1 

    RI = session_result/total_session
    
    #Add data to dataframe 
    k10 = user_info.loc[user_info['Your last.fm username'] == user_name]['K-10'].values[0]
    track_data = np.array([user_name,k10,total_session,total_tracks,RI])
    track_data=np.concatenate((track_data, avg_features),axis=0)
    all_user_summary.loc[len(all_user_summary)] = track_data
all_user_summary.to_csv('/content/drive/MyDrive/MMT_Project_Data/DurCovidData/all_user_summary.csv',index=False,encoding='utf-8')   

Processing file 1: 05157feb07f9e780637ffb96ae549365_session.csv...
Processing file 2: 06bd850533a713bb7800766860dc6e6e_session.csv...
Processing file 3: 03e7c2869742ac3b8b04a45675ab5b24_session.csv...
Processing file 4: 0538da9949fdaa2da6e5f18c0a27e146_session.csv...
Processing file 5: 03c38064fb423b47d5292bffc09785cc_session.csv...
Processing file 6: 05110aecc712b5c4d62efc592507dcce_session.csv...
Processing file 7: 080718a2684221573b174563e37c8f42_session.csv...
Processing file 8: 082ae734bd0f8399771b841e3f19f8a9_session.csv...
Processing file 9: 056d099110249e4dfc60cfc330f59f76_session.csv...
Processing file 10: 03a050d649de4bf6e489a064515e2f43_session.csv...
Processing file 11: 0274a823eec1e677645b20883a5e7897_session.csv...
Processing file 12: 04e3becfa6b8843aabcfed4e128ffff5_session.csv...
Processing file 13: 009327643adf158a3693b76e2edde04e_session.csv...
Processing file 14: 0820db544692f5180aeea5b7233c0f43_session.csv...
Processing file 15: 026c353028eefcd77442b35ce99832e0_sess

In [ ]:
#Save the file
all_user_summary.to_csv('/content/drive/MyDrive/MMT_Project_Data/DurCovidData/all_user_summary.csv',index=False,encoding='utf-8')

In [ ]:
all_user_summary.tail()
#all_user_summary.drop(all_user_summary.tail(1).index,inplace=True)

,user_id,K10,total_session,total_tracks,repetitiveness_index,danceability,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,QPS_Q1,QPS_Q2,QPS_Q3,QPS_Q4
480,fdff0438dc338ba560f4d3b9bdedc4e3,14,262,3350,0.049503731930103614,0.481669,-10.376715,0.645970,0.079353,0.388005,0.333626,0.179774,120.144289,0.213731,0.354328,0.388358,0.043582
481,fed309c0f4f060ea787a67157f0e6a77,16,383,3770,0.023728538805150873,0.547758,-8.857262,0.784085,0.054771,0.405313,0.048805,0.199491,121.611213,0.437931,0.171088,0.252785,0.138196
482,ff1d102d1eaef7a14d9404a16e869e78,31,451,8633,1.8088163123256027,0.530444,-10.434797,0.621105,0.110482,0.477129,0.181172,0.192644,117.311342,0.223908,0.243716,0.457547,0.074829
483,ff7aefe14748d4cfed2a7f6cbc6cb226,28,454,3000,0.29771712790852944,0.543937,-7.502812,0.421667,0.050961,0.249239,0.100887,0.190929,117.830498,0.321000,0.416667,0.234333,0.028000
484,fc9baa2bd54e1aed22218c60f6a831e7,19,489,7652,0.07345125394643742,0.370430,-5.999008,0.614219,0.123380,0.035828,0.545862,0.237804,124.545150,0.035024,0.921589,0.041166,0.002222


### Dynamic Feature extraction for all users and save in csv files (all_user_variability and all_user_autocorr).

In [ ]:
#Compute Autocorrelation for each column in dataframe
def get_df_autocorr(df, lag=1, axis=0):
    return df.apply(lambda col: col.autocorr(lag), axis=axis)

In [ ]:
feature_columns = ['danceability','loudness','mode','speechiness','acousticness','instrumentalness',
           'liveness','tempo','QPS_Q1','QPS_Q2','QPS_Q3','QPS_Q4']

In [ ]:
count=0
all_user_variability = pd.DataFrame(columns=['user_id','K-10','danceability',
                                         'loudness','mode','speechiness','acousticness','instrumentalness','liveness',
                                         'tempo','QPS_Q1','QPS_Q2','QPS_Q3','QPS_Q4'])
all_user_autocorr = pd.DataFrame(columns=['user_id','K-10','danceability',
                                         'loudness','mode','speechiness','acousticness','instrumentalness','liveness',
                                         'tempo','QPS_Q1','QPS_Q2','QPS_Q3','QPS_Q4'])
for file in os.listdir(session_folder):
    count+=1
    print(f'Processing file {count}: {file}...')
    user_name = file.split('_')[0]
    session_df = pd.read_csv(f'{session_folder}/{user_name}_session.csv',encoding='utf-8')
    auto_corr_user = get_df_autocorr(session_df[feature_columns]).values
    std_user = session_df[feature_columns].std().values
    k10 = user_info.loc[user_info['Your last.fm username'] == user_name]['K-10'].values[0]
    auto_corr_user = [user_name,k10] + auto_corr_user.tolist()
    std_user =[user_name,k10] +std_user.tolist()
    
    all_user_variability.loc[len(all_user_variability)] = std_user
    all_user_autocorr.loc[len(all_user_autocorr)] = auto_corr_user
all_user_autocorr= all_user_autocorr.fillna(0)
all_user_variability.to_csv('/content/drive/MyDrive/MMT_Project_Data/DurCovidData/all_user_variability.csv',index=False,encoding='utf-8') 
all_user_autocorr.to_csv('/content/drive/MyDrive/MMT_Project_Data/DurCovidData/all_user_autocorr.csv',index=False,encoding='utf-8') 

Processing file 1: 05157feb07f9e780637ffb96ae549365_session.csv...
Processing file 2: 06bd850533a713bb7800766860dc6e6e_session.csv...
Processing file 3: 03e7c2869742ac3b8b04a45675ab5b24_session.csv...
Processing file 4: 0538da9949fdaa2da6e5f18c0a27e146_session.csv...
Processing file 5: 03c38064fb423b47d5292bffc09785cc_session.csv...
Processing file 6: 05110aecc712b5c4d62efc592507dcce_session.csv...
Processing file 7: 080718a2684221573b174563e37c8f42_session.csv...
Processing file 8: 082ae734bd0f8399771b841e3f19f8a9_session.csv...
Processing file 9: 056d099110249e4dfc60cfc330f59f76_session.csv...
Processing file 10: 03a050d649de4bf6e489a064515e2f43_session.csv...
Processing file 11: 0274a823eec1e677645b20883a5e7897_session.csv...
Processing file 12: 04e3becfa6b8843aabcfed4e128ffff5_session.csv...
Processing file 13: 009327643adf158a3693b76e2edde04e_session.csv...
Processing file 14: 0820db544692f5180aeea5b7233c0f43_session.csv...
Processing file 15: 026c353028eefcd77442b35ce99832e0_sess

In [ ]:
all_user_autocorr.head()

,user_id,K-10,danceability,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,QPS_Q1,QPS_Q2,QPS_Q3,QPS_Q4
0,05157feb07f9e780637ffb96ae549365,38,0.036753,0.012497,0.092193,0.198493,0.032450,0.126769,-0.008440,0.071468,0.087669,0.017955,0.121224,0.019101
1,06bd850533a713bb7800766860dc6e6e,22,0.200379,0.162489,0.049703,0.155538,0.091320,0.142868,0.219916,0.070931,-0.019023,0.057256,0.050773,0.146999
2,03e7c2869742ac3b8b04a45675ab5b24,12,0.212409,0.192599,0.088977,0.194389,0.285146,0.200652,0.049879,0.172934,0.259115,0.082721,0.161561,0.162826
3,0538da9949fdaa2da6e5f18c0a27e146,30,0.139427,0.155880,0.012925,0.044748,0.127531,0.047814,0.056455,-0.005651,0.024583,0.050533,0.112787,0.017857
4,03c38064fb423b47d5292bffc09785cc,19,0.139628,0.195127,0.163588,0.048842,0.090204,0.085517,0.138682,0.039550,0.148056,0.063424,-0.001909,0.185546


In [ ]:
all_user_variability.head()

,user_id,K-10,danceability,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,QPS_Q1,QPS_Q2,QPS_Q3,QPS_Q4
0,05157feb07f9e780637ffb96ae549365,38,0.105606,2.689329,0.220432,0.043154,0.174171,0.217813,0.071304,11.751355,0.213665,0.277740,0.179081,0.127777
1,06bd850533a713bb7800766860dc6e6e,22,0.101556,2.618071,0.258939,0.033227,0.168973,0.182093,0.168618,13.808500,0.292232,0.324589,0.198771,0.153562
2,03e7c2869742ac3b8b04a45675ab5b24,12,0.081767,2.645149,0.146357,0.050171,0.154996,0.198674,0.050484,9.643929,0.144777,0.150591,0.219181,0.097228
3,0538da9949fdaa2da6e5f18c0a27e146,30,0.117242,2.766052,0.236219,0.075992,0.199739,0.183923,0.080559,15.382950,0.270806,0.261411,0.240458,0.088547
4,03c38064fb423b47d5292bffc09785cc,19,0.117222,3.070857,0.267048,0.036650,0.198486,0.239705,0.131624,14.915816,0.287767,0.306521,0.253601,0.122421
